# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 12:16PM,255110,10,MSP217578,"Methapharm, Inc.",Released
1,Jan 16 2023 12:16PM,255110,10,MSP217579,"Methapharm, Inc.",Released
2,Jan 16 2023 12:16PM,255110,10,MSP217581,"Methapharm, Inc.",Released
3,Jan 16 2023 12:16PM,255110,10,MSP217582,"Methapharm, Inc.",Released
4,Jan 16 2023 12:16PM,255110,10,MSP217583,"Methapharm, Inc.",Released
5,Jan 16 2023 12:16PM,255110,10,MSP217584,"Methapharm, Inc.",Released
6,Jan 16 2023 12:16PM,255110,10,MSP217585,"Methapharm, Inc.",Released
7,Jan 16 2023 12:16PM,255110,10,MSP217572,"Methapharm, Inc.",Released
8,Jan 16 2023 12:16PM,255110,10,MSP217573,"Methapharm, Inc.",Released
9,Jan 16 2023 12:16PM,255110,10,MSP217574,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,255105,Released,12
20,255107,Released,1
21,255108,Released,2
22,255110,Released,12
23,255111,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
255105,NaN,NaN,12.0
255107,NaN,NaN,1.0
255108,NaN,NaN,2.0
255110,NaN,NaN,12.0
255111,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255006,5.0,43.0,4.0
255049,0.0,0.0,42.0
255050,0.0,0.0,16.0
255053,0.0,0.0,1.0
255076,0.0,0.0,46.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255006,5,43,4
255049,0,0,42
255050,0,0,16
255053,0,0,1
255076,0,0,46


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255006,5,43,4
1,255049,0,0,42
2,255050,0,0,16
3,255053,0,0,1
4,255076,0,0,46


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255006,5,43,4
1,255049,,,42
2,255050,,,16
3,255053,,,1
4,255076,,,46


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 12:16PM,255110,10,"Methapharm, Inc."
12,Jan 16 2023 12:16PM,255111,10,"Methapharm, Inc."
13,Jan 16 2023 12:16PM,255111,10,Methapharm-G
14,Jan 16 2023 12:09PM,255108,10,Emerginnova
16,Jan 16 2023 12:07PM,255105,10,ISDIN Corporation
28,Jan 16 2023 12:07PM,255107,19,"GCH Granules USA, Inc."
29,Jan 16 2023 12:06PM,255097,10,ISDIN Corporation
73,Jan 16 2023 11:57AM,255104,10,ISDIN Corporation
74,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc."
75,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 16 2023 12:16PM,255110,10,"Methapharm, Inc.",,,12
1,Jan 16 2023 12:16PM,255111,10,"Methapharm, Inc.",,,2
2,Jan 16 2023 12:16PM,255111,10,Methapharm-G,,,2
3,Jan 16 2023 12:09PM,255108,10,Emerginnova,,,2
4,Jan 16 2023 12:07PM,255105,10,ISDIN Corporation,,,12
5,Jan 16 2023 12:07PM,255107,19,"GCH Granules USA, Inc.",,,1
6,Jan 16 2023 12:06PM,255097,10,ISDIN Corporation,,,44
7,Jan 16 2023 11:57AM,255104,10,ISDIN Corporation,,,1
8,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",,,1
9,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 12:16PM,255110,10,"Methapharm, Inc.",12,,
1,Jan 16 2023 12:16PM,255111,10,"Methapharm, Inc.",2,,
2,Jan 16 2023 12:16PM,255111,10,Methapharm-G,2,,
3,Jan 16 2023 12:09PM,255108,10,Emerginnova,2,,
4,Jan 16 2023 12:07PM,255105,10,ISDIN Corporation,12,,
5,Jan 16 2023 12:07PM,255107,19,"GCH Granules USA, Inc.",1,,
6,Jan 16 2023 12:06PM,255097,10,ISDIN Corporation,44,,
7,Jan 16 2023 11:57AM,255104,10,ISDIN Corporation,1,,
8,Jan 16 2023 11:14AM,255096,22,"NBTY Global, Inc.",1,,
9,Jan 16 2023 11:10AM,255093,10,Eywa Pharma Inc.,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 12:16PM,255110,10,"Methapharm, Inc.",12,,
1,Jan 16 2023 12:16PM,255111,10,"Methapharm, Inc.",2,,
2,Jan 16 2023 12:16PM,255111,10,Methapharm-G,2,,
3,Jan 16 2023 12:09PM,255108,10,Emerginnova,2,,
4,Jan 16 2023 12:07PM,255105,10,ISDIN Corporation,12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 12:16PM,255110,10,"Methapharm, Inc.",12.0,NaN,NaN
1,Jan 16 2023 12:16PM,255111,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Jan 16 2023 12:16PM,255111,10,Methapharm-G,2.0,NaN,NaN
3,Jan 16 2023 12:09PM,255108,10,Emerginnova,2.0,NaN,NaN
4,Jan 16 2023 12:07PM,255105,10,ISDIN Corporation,12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 16 2023 12:16PM,255110,10,"Methapharm, Inc.",12.0,0.0,0.0
1,Jan 16 2023 12:16PM,255111,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Jan 16 2023 12:16PM,255111,10,Methapharm-G,2.0,0.0,0.0
3,Jan 16 2023 12:09PM,255108,10,Emerginnova,2.0,0.0,0.0
4,Jan 16 2023 12:07PM,255105,10,ISDIN Corporation,12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3316263,204.0,3.0,0.0
15,765105,62.0,43.0,5.0
19,510197,48.0,3.0,0.0
21,255087,0.0,1.0,0.0
22,510149,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3316263,204.0,3.0,0.0
1,15,765105,62.0,43.0,5.0
2,19,510197,48.0,3.0,0.0
3,21,255087,0.0,1.0,0.0
4,22,510149,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,204.0,3.0,0.0
1,15,62.0,43.0,5.0
2,19,48.0,3.0,0.0
3,21,0.0,1.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,204.0
1,15,Released,62.0
2,19,Released,48.0
3,21,Released,0.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Completed,0.0,5.0,0.0,0.0,0.0
Executing,3.0,43.0,3.0,1.0,0.0
Released,204.0,62.0,48.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Completed,0.0,5.0,0.0,0.0,0.0
1,Executing,3.0,43.0,3.0,1.0,0.0
2,Released,204.0,62.0,48.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Completed,0.0,5.0,0.0,0.0,0.0
1,Executing,3.0,43.0,3.0,1.0,0.0
2,Released,204.0,62.0,48.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()